# LangGraph の基礎


In [ ]:
from dotenv import load_dotenv

load_dotenv(dotenv_path="../.env", override=True)

## 単純なチャットボットの実装


In [ ]:
from typing import Annotated
from typing_extensions import TypedDict

from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages


class State(TypedDict):
    messages: Annotated[list[BaseMessage], add_messages]

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4.1-nano")


def llm_node(state: State):
    ai_message = llm.invoke(state["messages"])
    return {"messages": [ai_message]}

In [ ]:
from langgraph.graph import StateGraph, START, END

graph_builder = StateGraph(State)
graph_builder.add_node("llm_node", llm_node)

graph_builder.add_edge(START, "llm_node")
graph_builder.add_edge("llm_node", END)

graph = graph_builder.compile()

In [ ]:
from IPython.display import Image, display

display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
from langchain_core.messages import HumanMessage

initial_state = {"messages": HumanMessage("こんにちは！")}

In [ ]:
graph.invoke(initial_state)

## 単純なエージェントの実装


In [ ]:
from typing import Annotated
from typing_extensions import TypedDict

from langgraph.graph.message import add_messages


class State(TypedDict):
    messages: Annotated[list[BaseMessage], add_messages]

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults

llm = ChatOpenAI(model="gpt-4.1-nano")
tools = [TavilySearchResults()]
llm_with_tools = llm.bind_tools(tools)


def llm_node(state: State):
    ai_message = llm_with_tools.invoke(state["messages"])
    return {"messages": [ai_message]}

In [ ]:
import json

from langchain_core.messages import ToolMessage
from langchain_core.tools import BaseTool


class BasicToolNode:
    def __init__(self, tools: list[BaseTool]) -> None:
        # {"ツール名": "ツール"} というdictを作成
        tools_by_name = {}
        for tool in tools:
            tools_by_name[tool.name] = tool
        self.tools_by_name = tools_by_name

    def __call__(self, state: State):
        latest_message = state["messages"][-1]

        tool_messages = []
        for tool_call in latest_message.tool_calls:
            tool = self.tools_by_name[tool_call["name"]]
            tool_result = tool.invoke(tool_call["args"])
            tool_messages.append(
                ToolMessage(
                    content=json.dumps(tool_result),
                    name=tool_call["name"],
                    tool_call_id=tool_call["id"],
                )
            )
        return {"messages": tool_messages}


tool_node = BasicToolNode(tools=tools)

In [ ]:
from langgraph.graph import StateGraph, START, END


graph_builder = StateGraph(State)
graph_builder.add_node("llm_node", llm_node)
graph_builder.add_node("tool_node", tool_node)


def is_last_message_tool_call(state: State):
    last_message = state["messages"][-1]
    if hasattr(last_message, "tool_calls") and len(last_message.tool_calls) > 0:
        return True
    return False


graph_builder.add_edge(START, "llm_node")
graph_builder.add_conditional_edges(
    "llm_node",
    is_last_message_tool_call,
    {
        True: "tool_node",
        False: END,
    },
)
graph_builder.add_edge("tool_node", "llm_node")
graph = graph_builder.compile()

In [ ]:
from IPython.display import Image, display

display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
from langchain_core.messages import HumanMessage

initial_state = {"messages": HumanMessage("こんにちは！")}
graph.invoke(initial_state)

In [ ]:
from langchain_core.messages import HumanMessage

initial_state = {"messages": HumanMessage("東京の今日の天気は？")}
graph.invoke(initial_state)

In [ ]:
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage

initial_state = {"messages": HumanMessage("東京の今日の天気は？")}

for event in graph.stream(initial_state, stream_mode="updates"):
    for value in event.values():
        latest_message = value["messages"][-1]
        if isinstance(latest_message, AIMessage):
            if (
                hasattr(latest_message, "tool_calls")
                and len(latest_message.tool_calls) > 0
            ):
                for tool_call in latest_message.tool_calls:
                    print(
                        f"Tool call: name = {tool_call['name']}, args = {tool_call['args']}"
                    )
            else:
                print(f"AI: {latest_message.content}")
        elif isinstance(latest_message, ToolMessage):
            print(f"Tool result: {latest_message.content}")
        else:
            print(latest_message)


## Q&A アプリケーション


In [ ]:
ROLES = {
    "1": {
        "name": "一般知識エキスパート",
        "description": "幅広い分野の一般的な質問に答える",
        "details": "幅広い分野の一般的な質問に対して、正確で分かりやすい回答を提供してください。",
    },
    "2": {
        "name": "生成AI製品エキスパート",
        "description": "生成AIや関連製品、技術に関する専門的な質問に答える",
        "details": "生成AIや関連製品、技術に関する専門的な質問に対して、最新の情報と深い洞察を提供してください。",
    },
    "3": {
        "name": "カウンセラー",
        "description": "個人的な悩みや心理的な問題に対してサポートを提供する",
        "details": "個人的な悩みや心理的な問題に対して、共感的で支援的な回答を提供し、可能であれば適切なアドバイスも行ってください。",
    },
}

In [ ]:
import operator
from typing import Annotated
from typing_extensions import TypedDict


class State(TypedDict):
    query: str
    current_role: str
    messages: Annotated[list[str], operator.add]
    current_judge: bool
    judgement_reason: str

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.runnables import ConfigurableField

llm = ChatOpenAI(model="gpt-4.1", temperature=0.0)
# 後からmax_tokensの値を変更できるように、変更可能なフィールドを宣言
llm = llm.configurable_fields(max_tokens=ConfigurableField(id="max_tokens"))

In [ ]:
from typing import Any

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser


def selection_node(state: State) -> dict[str, Any]:
    query = state["query"]
    role_options = "\n".join(
        [f"{k}. {v['name']}: {v['description']}" for k, v in ROLES.items()]
    )
    prompt = ChatPromptTemplate.from_template(
        """質問を分析し、最も適切な回答担当ロールを選択してください。

選択肢:
{role_options}

回答は選択肢の番号（1、2、または3）のみを返してください。

質問: {query}
""".strip()
    )
    # 選択肢の番号のみを返すことを期待したいため、max_tokensの値を1に変更
    chain = (
        prompt | llm.with_config(configurable=dict(max_tokens=1)) | StrOutputParser()
    )
    role_number = chain.invoke({"role_options": role_options, "query": query})

    selected_role = ROLES[role_number.strip()]["name"]
    return {"current_role": selected_role}

In [ ]:
def answering_node(state: State) -> dict[str, Any]:
    query = state["query"]
    role_name = state["current_role"]
    role = next(filter(lambda x: x["name"] == role_name, ROLES.values()))
    role_detail = role["details"]
    prompt = ChatPromptTemplate.from_template(
        """あなたは{role_name}として回答してください。以下の質問に対して、あなたの役割に基づいた適切な回答を提供してください。

役割の詳細:
{role_detail}

質問: {query}

回答:""".strip()
    )
    chain = prompt | llm | StrOutputParser()
    answer = chain.invoke(
        {"role_name": role_name, "role_detail": role_detail, "query": query}
    )
    return {"messages": [answer]}

In [ ]:
from pydantic import BaseModel, Field


class Judgement(BaseModel):
    reason: str = Field(description="判定理由")
    judge: bool = Field(description="判定結果")


def check_node(state: State) -> dict[str, Any]:
    query = state["query"]
    answer = state["messages"][-1]
    prompt = ChatPromptTemplate.from_template(
        """以下の回答の品質をチェックし、問題がある場合は'False'、問題がない場合は'True'を回答してください。
また、その判断理由も説明してください。

ユーザーからの質問: {query}
回答: {answer}
""".strip()
    )
    chain = prompt | llm.with_structured_output(Judgement)
    result: Judgement = chain.invoke({"query": query, "answer": answer})

    return {"current_judge": result.judge, "judgement_reason": result.reason}

In [ ]:
from langgraph.graph import StateGraph

workflow = StateGraph(State)

In [ ]:
workflow.add_node("selection", selection_node)
workflow.add_node("answering", answering_node)
workflow.add_node("check", check_node)

In [ ]:
# selectionノードから処理を開始
workflow.set_entry_point("selection")

In [ ]:
# selectionノードからansweringノードへ
workflow.add_edge("selection", "answering")
# answeringノードからcheckノードへ
workflow.add_edge("answering", "check")

In [ ]:
from langgraph.graph import END

# checkノードから次のノードへの遷移に条件付きエッジを定義
# state.current_judgeの値がTrueならENDノードへ、Falseならselectionノードへ
workflow.add_conditional_edges(
    "check", lambda state: state["current_judge"], {True: END, False: "selection"}
)

In [ ]:
graph = workflow.compile()

In [ ]:
from IPython.display import Image, display

display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
initial_state = {
    "query": "生成AIについて教えてください",
    "current_role": "",
    "messages": [],
    "current_judge": False,
    "judgement_reason": "",
}
graph.invoke(initial_state)